# **Project Hands-on**

![](/Volumes/jpm/naval/csv/Screenshot 2025-02-25 at 5.15.48 PM.png)

In [0]:
%sql
use catalog jpm;
create schema if not exists subro_bronze;
create schema if not exists subro_silver;
create schema if not exists subro_gold;

In [0]:
df=spark.read.csv("/Volumes/jpm/subro/financial/Financial Sample.csv",header=True,inferSchema=True)

In [0]:
%sql
drop table jpm.subro_bronze.financials_sample;

In [0]:
df.write.mode("overwrite").saveAsTable("jpm.subro_bronze.financials_sample")

In [0]:
%sql
alter table jpm.subro_bronze.financials_sample add columns ingestion_dt date;

In [0]:
%sql
update jpm.subro_bronze.financials_sample set ingestion_dt = current_date();

In [0]:
%sql
select year,count(0) from jpm.subro_bronze.financials_sample group by year ;

In [0]:
%sql
select year, count(0) from (
select * from jpm.subro_bronze.financials_sample
group by segment, country, product, discount_band, units_sold, manufacturing_price, sale_price, gross_sales, discounts, sales, cogs, profit, date, month_number, month_name, year,ingestion_dt)
group by 1;

In [0]:
%sql
select count(0) from jpm.subro_bronze.financials_sample where year is null;

In [0]:
%sql
create or replace table jpm.subro_silver.financials_sample as
select * from jpm.subro_bronze.financials_sample
group by segment, country, product, discount_band, units_sold, manufacturing_price, sale_price, gross_sales, discounts, sales, cogs, profit, date, month_number, month_name, year, ingestion_dt;

In [0]:
%sql
select count(0) from jpm.subro_silver.financials_sample;

In [0]:
%sql
select segment, country, product, ltrim(rtrim(discount_band)) as discount_band, cast(ltrim(rtrim(replace(units_sold,'$',''))) as int) as units_sold,
cast(ltrim(rtrim(replace(manufacturing_price,'$',''))) as double) as manufacturing_price,
cast(ltrim(rtrim(replace(sale_price,'$',''))) as double) as sale_price,
cast(replace(ltrim(rtrim(gross_sales)),'$','') as int) as gross_sales,
cast(ltrim(rtrim(replace(discounts,'$',''))) as double) as discounts,
cast(ltrim(rtrim(replace(saless,'$',''))) as double) as saless,
cast(ltrim(rtrim(replace(cogs,'$',''))) as double) as cogs,
cast(ltrim(rtrim(replace(profit,'$',''))) as double) as profit
from jpm.subro_bronze.financials_sample;